In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, "../")

In [3]:
import torch.nn as nn
from torchsummary import summary

from data.input_transforms import InputTransform
from data.cifar import CIFAR
from data.dataloader import get_dataloader
from models.optimizer import optimizer, scheduler
from networks.twolayernet import TwolayerNet
from networks.convnet import ConvNet
from utils.viz import plot_multiple_quantities_by_time
from main import train, evaluate

In [4]:
def experiment(train_transforms, valid_transforms, network, num_epochs, batch_size, opt_args, sch_args=None):
    
    print(train_transforms)
    print(num_epochs)
    print(batch_size)
    print(opt_args)
    print(sch_args)
    
    # define the datasets
    train_transform = InputTransform(train_transforms)
    valid_transform = InputTransform(valid_transforms)
    train_dataset = CIFAR(root="../datasets/CIFAR-10/", mode="train", transform=train_transform)
    valid_dataset = CIFAR(root="../datasets/CIFAR-10/", mode="valid", transform=valid_transform)

    # obtain the train dataloader
    train_loader = get_dataloader(train_dataset, train=True, batch_size=batch_size, num_workers=0)
    valid_loader = get_dataloader(valid_dataset, train=False, batch_size=batch_size, num_workers=0)
    
    if network == "tln":
        net = TwolayerNet(num_inputs=3 * 32 * 32, num_hidden=512, num_classes=10)
    elif network == "cnn":
        net = ConvNet(in_channels=3, num_classes=10)
    else:
        raise ValueError()
    arch = type(net).__name__

    # define the optimizer and scheduler
    opt = optimizer(model_params=net.parameters(), **opt_args)
    sch = None
    if sch_args is not None:
        sch = scheduler(opt=opt, **sch_args)

    # define the loss function
    loss_fn = nn.CrossEntropyLoss()

    # train the model
    epochs, train_losses, train_metrics, valid_losses, valid_metrics = train(
        net, loss_fn, train_loader, valid_loader, num_epochs=num_epochs, opt=opt, sch=sch,
    )

    # plot training curves
    plot_multiple_quantities_by_time(
        quantities=[train_losses["loss"], valid_losses["loss"]],
        time=epochs,
        labels=["Train", "Validation"],
        title=f"{arch} Loss curves",
        show=True,
        save=True,
        save_path="../results/cnn_loss_plot_best.png",
        ylabel="Loss",
    )
    plot_multiple_quantities_by_time(
        quantities=[train_metrics["accuracy"], valid_metrics["accuracy"]],
        time=epochs,
        labels=["Train", "Validation"],
        title=f"{arch} Accuracy curves",
        show=True,
        save=True,
        save_path="../results/cnn_accu_plot_best.png",
        ylabel="Accuracy",
    )
    
    return net, loss_fn

In [ ]:
train_transforms = [
    {
        "name": "ToTensor",
        "args": {},
    },
    {
        "name": "Normalize",
        "args": {"mean": (0.5, 0.5, 0.5), "std": (0.5, 0.5, 0.5)},
    },
    {
        "name": "RandomAffine",
        "args": {"degrees": 10, "translate": (0.1, 0.1), "scale": (0.8, 1.2)},
    },
]
valid_transforms = [
    {
        "name": "ToTensor",
        "args": {},
    },
    {
        "name": "Normalize",
        "args": {"mean": (0.5, 0.5, 0.5), "std": (0.5, 0.5, 0.5)},
    },
]

network = "cnn"
num_epochs = 100
batch_size = 128
opt_args = {"name": "Adam", "lr": 1e-3}

sch_args = None
sch_args = {"name": "MultiStepLR", "milestones": [50], "gamma": 0.1}

net, loss_fn = experiment(
    train_transforms, valid_transforms, network, num_epochs, batch_size, opt_args, sch_args=sch_args,
)

[{'name': 'ToTensor', 'args': {}}, {'name': 'Normalize', 'args': {'mean': (0.5, 0.5, 0.5), 'std': (0.5, 0.5, 0.5)}}, {'name': 'RandomAffine', 'args': {'degrees': 10, 'translate': (0.1, 0.1), 'scale': (0.8, 1.2)}}]
100
128
{'name': 'Adam', 'lr': 0.001}
{'name': 'MultiStepLR', 'milestones': [50], 'gamma': 0.1}
Selecting 40000 for train.
Selecting 10000 for valid.


Evaluate: Epoch [1/100]: 100%|██████████████████████████████| 79/79 [00:00<00:00, 115.61it/s]                                              


VALID 	: Summary: Loss: 1.6925 Accuracy: 0.3817
TRAIN 	: Summary: Loss: 1.9289 Accuracy: 0.2925


Evaluate: Epoch [2/100]: 100%|██████████████████████████████| 79/79 [00:00<00:00, 111.43it/s]                                              


VALID 	: Summary: Loss: 1.5297 Accuracy: 0.4442
TRAIN 	: Summary: Loss: 1.6867 Accuracy: 0.3843


...

Evaluate: Epoch [78/100]: 100%|██████████████████████████████| 79/79 [00:00<00:00, 121.53it/s]                                             


VALID 	: Summary: Loss: 0.9125 Accuracy: 0.6766
TRAIN 	: Summary: Loss: 0.9168 Accuracy: 0.6763


Evaluate: Epoch [79/100]: 100%|██████████████████████████████| 79/79 [00:00<00:00, 121.50it/s]                                             


VALID 	: Summary: Loss: 0.9163 Accuracy: 0.6771
TRAIN 	: Summary: Loss: 0.9196 Accuracy: 0.6724


Training: Epoch [80/100]:  35%|██████████▎                   | 108/313 [00:03<00:06, 31.29it/s]                                            

In [ ]:
summary(net, input_size=(3, 32, 32))

In [ ]:
test_transforms = [
    {
        "name": "ToTensor",
        "args": {},
    },
    {
        "name": "Normalize",
        "args": {"mean": (0.5, 0.5, 0.5), "std": (0.5, 0.5, 0.5)},
    },
]
test_transform = InputTransform(test_transforms)
test_dataset = CIFAR(root="../datasets/CIFAR-10/", mode="test", transform=test_transform)
test_loader = get_dataloader(test_dataset, train=False, batch_size=128, num_workers=0)

In [ ]:
evaluate(net, test_loader, loss_fn, 1, 1, "test")